In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Sample dataset
data = {
    'user': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Alice', 'Charlie', 'Bob', 'Charlie', 'Alice'],
    'item': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'C', 'D', 'E'],
    'rating': [5, 4, 4, 3, 2, 4, 5, 1, 2, 5]
}

# Create DataFrame
df = pd.DataFrame(data)

# Encode user and item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user'])
df['item_id'] = item_encoder.fit_transform(df['item'])

# Create user-item matrix
user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity,
                                  index=user_encoder.classes_,
                                  columns=user_encoder.classes_)

# Show similarity matrix
print("User Similarity Matrix (Cosine Similarity):\n")
print(user_similarity_df.round(2))

# Recommendation section
target_user = 'Bob'
target_user_id = user_encoder.transform([target_user])[0]

# Get similarity scores and sort
similar_users = user_similarity[target_user_id]
similar_users_indices = similar_users.argsort()[::-1][1:]  # Exclude self

# Try recommending from top similar users
recommendations = pd.Series(dtype='float64')

for sim_user_id in similar_users_indices:
    sim_user_ratings = user_item_matrix.iloc[sim_user_id]
    already_rated = user_item_matrix.iloc[target_user_id] > 0
    unseen_ratings = sim_user_ratings[~already_rated]

    if not unseen_ratings.empty:
        recommendations = unseen_ratings.sort_values(ascending=False)
        break

# Show recommendation results
if not recommendations.empty:
    recommended_items = item_encoder.inverse_transform(recommendations.index[:3])
    print(f"\nTop Recommendations for {target_user}: {list(recommended_items)}")
else:
    print(f"\nNo recommendations available for {target_user} (may have rated all items).")

User Similarity Matrix (Cosine Similarity):

         Alice   Bob  Charlie
Alice     1.00  0.76     0.69
Bob       0.76  1.00     0.68
Charlie   0.69  0.68     1.00

Top Recommendations for Bob: ['E', 'D']
